# GROUP BY

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.options.display.float_format = '{:_.3f}'.format

In [3]:
df = sns.load_dataset('diamonds')
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.230,Ideal,E,SI2,61.500,55.000,326,3.950,3.980,2.430
1,0.210,Premium,E,SI1,59.800,61.000,326,3.890,3.840,2.310
2,0.230,Good,E,VS1,56.900,65.000,327,4.050,4.070,2.310
3,0.290,Premium,I,VS2,62.400,58.000,334,4.200,4.230,2.630
4,0.310,Good,J,SI2,63.300,58.000,335,4.340,4.350,2.750
...,...,...,...,...,...,...,...,...,...,...
53935,0.720,Ideal,D,SI1,60.800,57.000,2757,5.750,5.760,3.500
53936,0.720,Good,D,SI1,63.100,55.000,2757,5.690,5.750,3.610
53937,0.700,Very Good,D,SI1,62.800,60.000,2757,5.660,5.680,3.560
53938,0.860,Premium,H,SI2,61.000,58.000,2757,6.150,6.120,3.740


In [4]:
df.groupby('cut').mean()

,carat,depth,table,price,x,y,z
cut,,,,,,,
Ideal,0.703,61.709,55.952,3_457.542,5.507,5.520,3.401
Premium,0.892,61.265,58.746,4_584.258,5.974,5.945,3.647
Very Good,0.806,61.818,57.956,3_981.760,5.741,5.770,3.560
Good,0.849,62.366,58.695,3_928.864,5.839,5.851,3.640
Fair,1.046,64.042,59.054,4_358.758,6.247,6.183,3.983


In [5]:
df.groupby('cut')['carat'].count()

cut
Ideal        21551
Premium      13791
Very Good    12082
Good          4906
Fair          1610
Name: carat, dtype: int64

## Recorrer los grupos con un for

In [6]:
for key, group in df.groupby('cut'):
    price = group['price'].mean()
    print('Cut: {} \t Price: {}'.format(key,price))

Cut: Ideal 	 Price: 3457.541970210199
Cut: Premium 	 Price: 4584.2577042999055
Cut: Very Good 	 Price: 3981.7598907465654
Cut: Good 	 Price: 3928.864451691806
Cut: Fair 	 Price: 4358.757763975155


## Convirtiendo a DataFrame

In [7]:
# df.groupby(['cut','color'])['price'].mean().to_frame()
df.groupby(['cut','color'])['price'].aggregate(['min',np.mean,'max']).head(10)

min      mean    max
cut     color                      
Ideal   D      367 2_629.095  18693
        E      326 2_597.550  18729
        F      408 3_374.939  18780
        G      361 3_720.706  18806
        H      357 3_889.335  18760
        I      348 4_451.970  18779
        J      340 4_918.186  18508
Premium D      367 3_631.293  18575
        E      326 3_538.914  18477
        F      342 4_324.890  18791

### Aplicando distintas funciones a distintas columnas

In [8]:
def mean_kilo(x):
    return np.mean(x)/1000

In [9]:
dict_agg = {'carat':[min, max],'price':[np.mean, mean_kilo]}
dict_agg

{'carat': [<function min>, <function max>],
 'price': [<function numpy.mean(a, axis=None, dtype=None, out=None, keepdims=<no value>, *, where=<no value>)>,
  <function __main__.mean_kilo(x)>]}

In [15]:
# df.groupby(['cut','color']).apply(mean_kilo)
# df.groupby(['cut','color']).apply(lambda x : np.mean(x)/1000)
df.groupby(['cut','color']).aggregate(dict_agg).head(10)

carat           price          
                min   max      mean mean_kilo
cut     color                                
Ideal   D     0.200 2.750 2_629.095     2.629
        E     0.200 2.280 2_597.550     2.598
        F     0.230 2.450 3_374.939     3.375
        G     0.230 2.540 3_720.706     3.721
        H     0.230 3.500 3_889.335     3.889
        I     0.230 3.220 4_451.970     4.452
        J     0.230 3.010 4_918.186     4.918
Premium D     0.200 2.570 3_631.293     3.631
        E     0.200 3.050 3_538.914     3.539
        F     0.200 3.010 4_324.890     4.325

## Aplicando filtros

In [11]:
def filtro(x):
    return mean_kilo(x['price']) > 4

In [12]:
df.groupby('cut').filter(filtro)

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.210,Premium,E,SI1,59.800,61.000,326,3.890,3.840,2.310
3,0.290,Premium,I,VS2,62.400,58.000,334,4.200,4.230,2.630
8,0.220,Fair,E,VS2,65.100,61.000,337,3.870,3.780,2.490
12,0.220,Premium,F,SI1,60.400,61.000,342,3.880,3.840,2.330
14,0.200,Premium,E,SI2,60.200,62.000,345,3.790,3.750,2.270
...,...,...,...,...,...,...,...,...,...,...
53928,0.790,Premium,E,SI2,61.400,58.000,2756,6.030,5.960,3.680
53930,0.710,Premium,E,SI1,60.500,55.000,2756,5.790,5.740,3.490
53931,0.710,Premium,F,SI1,59.800,62.000,2756,5.740,5.730,3.430
53934,0.720,Premium,D,SI1,62.700,59.000,2757,5.690,5.730,3.580


In [13]:
df.groupby('cut').filter(filtro)['cut'].unique()

['Premium', 'Fair']
Categories (5, object): ['Ideal', 'Premium', 'Very Good', 'Good', 'Fair']